<h1>RSA - алгоритм</h1>

<h2>Проверка числа на простоту с использованием теста Миллера — Рабина</h2>

<p>
Уверждение:

Пусть $n$ — простое число и $n-1=2^sd$, где $d$ — нечётно. Тогда для любого $a$ из &#8484; выполняется хотя бы одно из условий:

    
1. $a^d ≡ 1\ (mod\ n)$
2. Существует целое число $r < s$ такое что $a^{2^{r}d} ≡ -1\ (mod\ n)$

    
Так как алгоритм является вероятностным, то следует отметить, что вероятность ошибки не превосходит $\frac{1}{4^k}$, где $k$ - количество чисел $a$, потенциально являющиеся свидетялями простоты. $k$ выбирается порядка $log_2(n)$
</p>

In [1]:
def IsPrime(n):
    if n == 1: return False
    
    import math, random
    
    # проверка на делимость в пределах 257 делителей
    for i in range(2, min(257, math.ceil(math.sqrt(n))) ):
        if n % i == 0:
            return False
    
    # тест Миллера — Рабина
    k = max(math.ceil(math.log(n, 2)), 10)
    
    s = 0
    d = n - 1
    
    while d % 2 == 0:
        s += 1
        d //= 2
    
    for _ in range(k):
        a_k = random.randint(2, n - 2)
        x = pow(a_k, d, n)
        
        if x == 1 or x == n - 1: # проверка на первое условие
            continue
        
        flag = False
        for _ in range(s - 1): # проверка на второе условие
            x = pow(x, 2, n)
            if x == 1:
                return False
            if x == n - 1:
                flag = True
                break
        
        if flag:
            continue
            
        return False
    return True

<h2>Генерация простого числа с заданным количеством бит</h2>

In [2]:
def GenPrimeNumber(k = 1024):
    import random
    
    bits = [random.choice([0, 1]) for _ in range(k)]
    
    bits[0], bits[-1] = 1, 1
    
    def ToNum(lst):
        k = 0
        result = 0

        for i in lst:
            result += i * (2 ** k)
            k += 1

        return result
    
    result = ToNum(bits)
    
    while not IsPrime(result):
        result += 2
        
    return result

<h2>Расширенный алгоритм Евклида для задачи $de ≡ 1\ (mod\ φ(n))$</h2>

In [3]:
def Inverse(e, phi_n):
    t = 0   
    newt = 1
    r = phi_n     
    newr = e

    while newr != 0:
        quotient = r // newr
        t, newt = newt, t - quotient * newt
        r, newr = newr, r - quotient * newr

    if r > 1:
        return "не обратимо"
    if t < 0:
        t = t + phi_n

    return t

<h2>RSA - алгоритм</h2>

In [4]:
def RSA(k = 1024):
    import random
    
    p = GenPrimeNumber(k)
    q = GenPrimeNumber(k)
    
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = random.choice([17, 257, 65537, GenPrimeNumber(10)])
    d = Inverse(e, phi_n)
    
    return [(e, n), (d, n)]

In [5]:
open_key, secret_key = RSA()
print(open_key)

(257, 11547392519580649211156652769808706886762256656509668165678521993230255233272719864671570677068135557589172463595636685463909095724006149462027507446682621733310072514846549449367574923643146790867742379075276234450579350747703862300246506084787469564152337746166401429058093312836230559064975976682403187046882796407301375324745177711284935001132248072019300331556577579331232203163335819640506992792542905395183582932254182699803492092029859147326587048134092579646761273519077304122256036895659898292963630450087169994481341424987639022246888628055123836564814065474296620972659671230527427798903521081878402355649)


<h2>Пример (плохой) шифрования</h2>

In [6]:
def Encrypt(message, open_key):
    import random
    
    lst = [random.randint(1, 1000)] + [ord(i) for i in message]
    
    a = 0
    
    for i in range(len(lst)):
        lst[i] = lst[i] + a
        a = lst[i]
    
    return [pow(i, *open_key) for i in lst]

def Decrypt(lst, secret_key):
    temp_lst = [pow(i, *secret_key) for i in lst]
    
    message = ""
    
    a = 0
    
    for i in range(len(lst)):
        message += chr(temp_lst[i] - a)
        a = temp_lst[i]
        
    return message[1:]

In [7]:
message = "Hellow, World!"
en = Encrypt(message, open_key)
Decrypt(en, secret_key)

'Hellow, World!'